In [11]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve

In [31]:
# path for data files
base_path="../data/"
order_path=base_path+'orders.csv'
product_path=base_path+'order_products__prior.csv'
order_clean_path=base_path+'order_clean'
product_clean_path=base_path+'product_clean'

In [16]:
# Reading order priors only
fields_order=["order_id","user_id"]
df_orders = pd.read_csv(order_path)
df_orders = df_orders.loc[df_orders.eval_set == "prior"][fields_order]
df_orders.head()

,order_id,user_id
0,2539329,1
1,2398795,1
2,473747,1
3,2254736,1
4,431534,1


In [17]:
# Reading products priors only
fields_product=["order_id","product_id"]
df_products = pd.read_csv(product_path)[fields_product]
df_products.head()

,order_id,product_id
0,2,33120
1,2,28985
2,2,9327
3,2,45918
4,2,30035


In [18]:
# User with less than 5 purchases # OUTLIER REMOVAL ~10% of the total users
df_count_user = df_orders[["user_id"]]
df_count_user = df_count_user.groupby(['user_id']).size().reset_index().rename(columns={0:'count'})
df_count_user.head()

,user_id,count
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [23]:
#Bottom Users 
df_bottom_users = df_count_user[df_count_user["count"] < 5][['user_id']]
remove_from_users = df_bottom_users['user_id'].tolist()
df_bottom_users.head()

,user_id
4,5
5,6
7,8
8,9
19,20


In [24]:
# Order Ids to Remove
order_id_index = df_orders[df_orders['user_id'].isin(remove_from_users)].order_id.tolist()

In [25]:
# Removing the users from orders
df_orders = df_orders[~df_orders['user_id'].isin(remove_from_users)]

In [26]:
# Removing the orders from products
df_products = df_products[~df_products['order_id'].isin(order_id_index)]

In [28]:
# Calculating Bottom Products less than 100 sold i.e bottom 20%
df_count_products = df_products[["product_id"]].groupby(['product_id']).size().reset_index().rename(columns={0:'count'})
bottom_products = df_count_products[df_count_products["count"] <=100]
remove_from_products = bottom_products['product_id'].tolist()

In [30]:
# Removing Bottom Products
df_products = df_products[~df_products['product_id'].isin(remove_from_products)]

In [34]:
# Saving the cleaned Products
df_products.to_pickle(product_clean_path)

In [35]:
# Saving the cleaned orders
df_orders.to_pickle(order_clean_path)